# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-81ee11a9b4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA seasons explorative search") 

Consider the following exploratory information need:

>  investigate the world of basketball. Compare french teams and US teams, country of citizenship of NBA players, and teams which won NBA seasons.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q41421`| Michael Jordan     | node |
| `wd:Q25369`| Kobe Bryant     | node |





Also consider

```
wd:Q41421 ?p ?obj .
```

is the BGP to retrieve all **properties of Michael Jordan**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

6. Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

8. Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

9. Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)


## Task 1
Identify the BGP for basketball player.

### Get the first 25 proprieties of "Micheal Jordan"  (wd:Q41421) that do not contain "ID".

In [262]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
   wd:Q41421 ?p ?obj .   
   ?p sc:name ?name .   
   FILTER REGEX(?name, "^((?!ID).)*$") .   
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1449'), ('name', 'nickname')]
[('p', 'http://www.wikidata.org/prop/direct/P1477'), ('name', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1618'), ('name', 'sport number')]
[

### Get the first 100 proprieties of "Micheal Jordan"  (wd:Q41421) that do not contain "ID" .

In [263]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
   wd:Q41421 ?p ?obj .   
   ?p sc:name ?name .   
   FILTER REGEX(?name, "^((?!ID).)*$") .   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1449'), ('name', 'nickname')]
[('p', 'http://www.wikidata.org/prop/direct/P1477'), ('name', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('name', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1618'), ('name', 'sport number')]
[

### Get the first 100 proprieties of "Micheal Jordan"  (wd:Q41421) that do contain "ID"

In [264]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
   wd:Q41421 ?p ?obj .   
   ?p sc:name ?name .   
   FILTER CONTAINS(?name, "ID") .   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1048'), ('name', 'NCL ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1315'), ('name', 'NLA Trove ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1375'), ('name', 'NSK ID')]
[('p', 'http://ww

### Get the fist 25 "profession" (wdt:P106) of the instance "Micheal Jordan" (wd:Q41421)

In [265]:
queryString = """
SELECT DISTINCT ?profession ?professionName
WHERE {   
   wd:Q41421 wdt:P106 ?profession .
   ?profession sc:name ?professionName .   
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('profession', 'http://www.wikidata.org/entity/Q131524'), ('professionName', 'entrepreneur')]
[('profession', 'http://www.wikidata.org/entity/Q18814623'), ('professionName', 'autobiographer')]
[('profession', 'http://www.wikidata.org/entity/Q33999'), ('professionName', 'actor')]
[('profession', 'http://www.wikidata.org/entity/Q3665646'), ('professionName', 'basketball player')]
[('profession', 'http://www.wikidata.org/entity/Q10871364'), ('professionName', 'baseball player')]
[('profession', 'http://www.wikidata.org/entity/Q17221'), ('professionName', 'spokesperson')]
6


### Get all data property of "Micheal Jordan" (wd:Q41421)

In [73]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q41421 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1006'), ('label', 'Nationale Thesaurus voor Auteurs ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('label', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1048'), ('label', 'NCL ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1207'), ('label', 'NUKAT ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('label', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1263'), ('label', 'NNDB people ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1266'), ('label', 'AlloCiné person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1280'), ('label', 'CONOR.SI ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1285'), ('label', 'Munzinger Sport number')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1315'), ('label', 'NL

### Get all data property of "Micheal Jordan"  (wd:Q41421) that do not contain ID

In [74]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q41421 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1285'), ('label', 'Munzinger Sport number')]
[('property', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('property', 'http://www.wikidata.org/prop/direct/P1477'), ('label', 'birth name')]
[('property', 'http://www.wikidata.org/prop/direct/P1559'), ('label', 'name in native language')]
[('property', 'http://www.wikidata.org/prop/direct/P1618'), ('label', 'sport number')]
[('property', 'http://www.wikidata.org/prop/direct/P2031'), ('label', 'work period (start)')]
[('property', 'http://www.wikidata.org/prop/direct/P2032'), ('label', 'work period (end)')]
[('property', 'http://www.wikidata.org/prop/direct/P2048'), ('label', 'height')]
[('property', 'http://www.wikidata.org/prop/direct/P2067'), ('label', 'mass')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P2561'), ('label', 'name')]
[('property', 'http://www.wikidata.org/

### Get the class of "Micheal Jordan" (wd:Q41421)

In [76]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q41421 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
1


### Get all object properties of "Micheal Jordan"  (wd:Q41421)

In [78]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q41421 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('label', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P118'), ('label', 'league')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('label', 'participant in')]
[('property', 'http://www.wikidata.org/prop/direct/P1411'), ('label', 'nominated for')]
[('property', 'http://www.wikidata.org/prop/direct/P1412'), ('label', 'languages spoken, written or signed')]
[('property', 'http://www.wikidata.org/prop/direct/P1532'), ('label', 'country for sport')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P172'), ('label', 'ethnic group')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1830'), ('label', 'owner of')]
[('property', 'http:

### Get all the classes connected to "Micheal Jordan"  (wd:Q41421)

In [79]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q41421 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q1046088'), ('label', 'hall of fame')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q337330'), ('label', 'Sportsperson of the Year')]
[('objectClass', 'http://www.wikidata.org/entity/Q20971972'), ('label', 'Original Public Ivy')]
[('objectClass', 'http://www.wikidata.org/entity/Q23002039'), ('label', 'public educational institution of the United States')]
[('objectClass', 'http://www.wikidata.org/entity/Q186165'), ('label', 'web portal')]
[('objectClass', 'http://www.wikidata.org/entity/Q618779'), ('label', 'award')]
[('objectClass', 'http://www.wikidata.org/entity/Q161928'), ('label', 'footwear')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q185

### Get all the couples (object property - list of the classes of the objects that are connected to "Micheal Jordan"  (wd:Q41421) through that object property)

In [80]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q41421 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12308941,http://www.wikidata.org/entity/Q12308941'), ('labelsObjectsClasses', 'male given name,male given name')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objectsClasses', 'http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q12737077,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q4164871,http://www.wikidata.org/entity/Q4164871,http://www.wikidata.org/entity/Q108300140'), ('labelsObjectsClasses', 'occupation,occupation,profession,profession,profession,profession,profession,position,position,occupation group according to ISCO')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('labe

### Get all the couples (object property - list of the  objects that are connected to "Micheal Jordan"  (wd:Q41421) through that object property)

In [81]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q41421 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P735'), ('labelProperty', 'given name'), ('objects', 'http://www.wikidata.org/entity/Q4927524,http://www.wikidata.org/entity/Q13028078'), ('labelsObjects', 'Michael,Jeffrey')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objects', 'http://www.wikidata.org/entity/Q11310897'), ('labelsObjects', 'Jordan Rules')]
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation'), ('objects', 'http://www.wikidata.org/entity/Q131524,http://www.wikidata.org/entity/Q18814623,http://www.wikidata.org/entity/Q33999,http://www.wikidata.org/entity/Q3665646,http://www.wikidata.org/entity/Q10871364,http://www.wikidata.org/entity/Q17221'), ('labelsObjects', 'entrepreneur,autobiographer,actor,basketball player,baseball player,spokesperson')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('labelProperty', 'participant in'), ('objects', 'http://www.wikidata.org/en

### Final Query

Final query for this task

In [261]:
queryString = """
SELECT DISTINCT ?occupation ?labelOccupation
WHERE 
{   
   wd:Q41421 wdt:P106 ?occupation .  
   # Labels
   ?occupation sc:name ?labelOccupation .      
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q131524'), ('labelOccupation', 'entrepreneur')]
[('occupation', 'http://www.wikidata.org/entity/Q18814623'), ('labelOccupation', 'autobiographer')]
[('occupation', 'http://www.wikidata.org/entity/Q33999'), ('labelOccupation', 'actor')]
[('occupation', 'http://www.wikidata.org/entity/Q3665646'), ('labelOccupation', 'basketball player')]
[('occupation', 'http://www.wikidata.org/entity/Q10871364'), ('labelOccupation', 'baseball player')]
[('occupation', 'http://www.wikidata.org/entity/Q17221'), ('labelOccupation', 'spokesperson')]
6


## Task 2
Identify the BGP for basketball.

### Get the first 25 proprieties of "Chicago Bulls" (wd:Q128109)

In [269]:
queryString = """
SELECT DISTINCT ?prop ?label
WHERE {   
   wd:Q128109  ?prop ?obj.   
   ?prop sc:name ?label.   
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P115'), ('label', 'home venue')]
[('prop', 'http://www.wikidata.org/prop/direct/P118'), ('label', 'league')]
[('prop', 'http://www.wikidata.org/prop/direct/P127'), ('label', 'owned by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P159'), ('label', 'headquarters location')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2397'), ('label', 'YouTube channel ID')]
[

### Get the first 25 proprieties of "basketball team" (wd:Q13393265)

In [270]:
queryString = """
SELECT DISTINCT ?prop ?label
WHERE {   
   wd:Q13393265  ?prop ?obj.   
   ?prop sc:name ?label.   
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('prop', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('prop', 'http://www.wikidata.org/prop/direct/P8408'), ('label', 'KBpedia ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P910'), ('label', "topic's main category")]
8


### Get the first 25 subjects and objects of the propriety "sport" (wdt:P641)

In [271]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj ?obj ?labelObj
WHERE {   
   ?sbj wdt:P641 ?obj .
   ?sbj sc:name ?labelSbj .
   ?obj sc:name ?labelObj .
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q2348425'), ('labelSbj', 'sports in Sweden'), ('obj', 'http://www.wikidata.org/entity/Q53121'), ('labelObj', 'cycling')]
[('sbj', 'http://www.wikidata.org/entity/Q19277056'), ('labelSbj', '2015 Canary Wharf Squash Classic'), ('obj', 'http://www.wikidata.org/entity/Q133201'), ('labelObj', 'squash')]
[('sbj', 'http://www.wikidata.org/entity/Q2856464'), ('labelSbj', 'Canary Wharf Squash Classic'), ('obj', 'http://www.wikidata.org/entity/Q133201'), ('labelObj', 'squash')]
[('sbj', 'http://www.wikidata.org/entity/Q4629184'), ('labelSbj', '2013 Canary Wharf Squash Classic'), ('obj', 'http://www.wikidata.org/entity/Q133201'), ('labelObj', 'squash')]
[('sbj', 'http://www.wikidata.org/entity/Q15844679'), ('labelSbj', '2014 FIM Motocross World Championship season'), ('obj', 'http://www.wikidata.org/entity/Q215862'), ('labelObj', 'motocross')]
[('sbj', 'http://www.wikidata.org/entity/Q15729533'), ('labelSbj', 'beach soccer in Estonia'), ('obj', 'ht

### Get the first 25 objects connected with "sport" (wdt:P641) to "basketball team" (wd:Q13393265)

In [272]:
queryString = """
SELECT DISTINCT ?obj ?label
WHERE {   
   wd:Q13393265  wdt:P641 ?obj.   
   ?obj sc:name ?label.   
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('label', 'basketball')]
1


### Get the class of "basketball" (wd:Q5372)

In [273]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q5372 wdt:P31 ?class.   
   ?class sc:name ?label.   
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('class', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
2


### Get the first 25 proprieties of "type of sport" (wd:Q31629)

In [274]:
queryString = """
SELECT DISTINCT ?prop ?label
WHERE {   
   wd:Q31629 ?prop ?obj .
   ?prop sc:name ?label
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2347'), ('label', 'YSO ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2671'), ('label', 'Google Knowledge Graph ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P910'), ('l

### Get the fist 25 subjects connected with "sport" (wdt:P641) to "basketball" (wd:Q5372)

In [275]:
queryString = """
SELECT DISTINCT ?basket ?label 
WHERE {   
   ?basket wdt:P641 wd:Q5372 ;
           sc:name ?label .
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('basket', 'http://www.wikidata.org/entity/Q16191059'), ('label', "2013–14 Atlantic Coast Conference men's basketball season")]
[('basket', 'http://www.wikidata.org/entity/Q16192074'), ('label', "2013–14 Southeastern Conference men's basketball season")]
[('basket', 'http://www.wikidata.org/entity/Q19863393'), ('label', "2008–09 Atlantic 10 Conference men's basketball season")]
[('basket', 'http://www.wikidata.org/entity/Q2348425'), ('label', 'sports in Sweden')]
[('basket', 'http://www.wikidata.org/entity/Q2811918'), ('label', '1947 in basketball')]
[('basket', 'http://www.wikidata.org/entity/Q2813083'), ('label', '1991 in basketball')]
[('basket', 'http://www.wikidata.org/entity/Q2813265'), ('label', '1996 in basketball')]
[('basket', 'http://www.wikidata.org/entity/Q2813922'), ('label', '2001 in basketball')]
[('basket', 'http://www.wikidata.org/entity/Q2855716'), ('label', '2012–13 ACB season')]
[('basket', 'http://www.wikidata.org/entity/Q2948730'), ('label', "2010 NCAA M

### Get the class of "basketball player" (wd:Q10871364)

In [276]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q10871364 wdt:P31 ?class .
   ?class sc:name ?label .
}
LIMIT 25
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
1


### Find first 25 proprieties of "basketball player" (wd:Q10871364)

In [277]:
queryString = """
SELECT DISTINCT ?prop ?label
WHERE {   
   wd:Q10871364 ?prop ?obj .
   ?prop sc:name ?label
}
LIMIT 25
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('prop', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('prop', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P425'), ('label', 'field of t

### Get all the objects connected with "drafted by" (wdt:P647) to "Micheal Jordan" (wd:Q41421)

In [266]:
queryString = """
SELECT DISTINCT ?uri ?label
WHERE {   
   wd:Q41421 wdt:P647 ?uri.   
   ?uri sc:name ?label .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('uri', 'http://www.wikidata.org/entity/Q128109'), ('label', 'Chicago Bulls')]
1


### Get all data property of "Chicago Bulls" (wd:Q128109)

In [267]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q128109 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2397'), ('label', 'YouTube channel ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3221'), ('label', 'New York Times topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4013'), ('label', 'Giphy username')]
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('lab

### Get all data property of "Chicago Bulls"  (wd:Q128109) that do not contain ID

In [88]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q128109 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1449'), ('label', 'nickname')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P4013'), ('label', 'Giphy username')]
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
[('property', 'http://www.wikidata.org/prop/direct/P8687'), ('label', 'social media followers')]
6


### Get the class of "Chicago Bulls" (wd:Q128109)

In [91]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q128109 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q13393265'), ('label', 'basketball team')]
1


### Get all object properties of "Chicago Bulls"  (wd:Q128109)

In [92]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q128109 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P115'), ('label', 'home venue')]
[('property', 'http://www.wikidata.org/prop/direct/P118'), ('label', 'league')]
[('property', 'http://www.wikidata.org/prop/direct/P127'), ('label', 'owned by')]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P159'), ('label', 'headquarters location')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P286'), ('label', 'head coach')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('label', 'part of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P822'), ('label', 'mascot')]
[

### Get all the classes connected to "Chicago Bulls"  (wd:Q128109)

In [93]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q128109 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q18536323'), ('label', 'basketball league')]
[('objectClass', 'http://www.wikidata.org/entity/Q15986354'), ('label', 'basketball association')]
[('objectClass', 'http://www.wikidata.org/entity/Q1093829'), ('label', 'city of the United States')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406554'), ('label', 'sports competition')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectClass', 'http://www.wikidata.org/entity/Q1549591'), ('label', 'big city')]
[('objectClass', 'http://www.wikidata.org/entity/Q15991290'), ('label', 'professional sports league')]
[('objectClass', 'http://www.wikidata.org/entity/Q1637706'), ('label', 'million city')]
[('objectClass', 'http://www.wikidata

### Get all the couples (object property - list of the classes of the objects that are connected to "Chicago Bulls"  (wd:Q128109) through that object property)

In [95]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q128109 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objectsClasses', 'http://www.wikidata.org/entity/Q13406554'), ('labelsObjectsClasses', 'sports competition')]
[('property', 'http://www.wikidata.org/prop/direct/P859'), ('labelProperty', 'sponsor'), ('objectsClasses', 'http://www.wikidata.org/entity/Q4830453,http://www.wikidata.org/entity/Q891723,http://www.wikidata.org/entity/Q46970'), ('labelsObjectsClasses', 'business,public company,airline')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objectsClasses', 'http://www.wikidata.org/entity/Q1489259,http://www.wikidata.org/entity/Q99541706,http://www.wikidata.org/entity/Q1520223,http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q512187,http://www.wikidata.org/entity/Q5255892,http://www.wikidata.org/entity/Q6256'), ('labelsObjectsClasses', 'superpower,historical unrecognized state,constitutional republic,sovereign state,federa

### Get all the couples (object property - list of the  objects that are connected to "Chicago Bulls"  (wd:Q128109) through that object property)

In [96]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q128109 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objects', 'http://www.wikidata.org/entity/Q745984'), ('labelsObjects', 'Central Division')]
[('property', 'http://www.wikidata.org/prop/direct/P859'), ('labelProperty', 'sponsor'), ('objects', 'http://www.wikidata.org/entity/Q174769,http://www.wikidata.org/entity/Q37189902'), ('labelsObjects', 'United Airlines,Mr. Submarine')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objects', 'http://www.wikidata.org/entity/Q30'), ('labelsObjects', 'United States of America')]
[('property', 'http://www.wikidata.org/prop/direct/P118'), ('labelProperty', 'league'), ('objects', 'http://www.wikidata.org/entity/Q155223'), ('labelsObjects', 'National Basketball Association')]
[('property', 'http://www.wikidata.org/prop/direct/P822'), ('labelProperty', 'mascot'), ('objects', 'http://www.wikidata.org/entity/Q4890070'), ('labelsObjects', 'Benny the Bull')]
[('prope

### Get all data property of "basketball team"  (wd:Q13393265) that do not contain ID

In [97]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q13393265 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P3417'), ('label', 'Quora topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P8408'), ('label', 'KBpedia ID')]
3


### Get all data property of "basketball team" (wd:Q13393265) that do not contain ID

In [98]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q13393265 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
1


### Get the class of "basketball team" (wd:Q13393265)

In [100]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q13393265 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get all object properties of "basketball team"  (wd:Q13393265)

In [101]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q13393265 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('label', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('label', "topic's main category")]
5


### Get all the classes connected to "basketball team"  (wd:Q13393265)

In [105]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q13393265 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
[('objectClass', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q4167836'), ('label', 'Wikimedia category')]
[('objectClass', 'http://www.wikidata.org/entity/Q16334295'), ('label', 'group of humans')]
[('objectClass', 'http://www.wikidata.org/entity/Q19887878'), ('label', 'Wikimedia infobox template')]
6


### Get all the couples (object property - list of the classes of the objects that are connected to "basketball team"  (wd:Q13393265) through that object property)

In [106]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q13393265 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q28640,http://www.wikidata.org/entity/Q16334295'), ('labelsObjectsClasses', 'profession,profession,group of humans')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objectsClasses', 'http://www.wikidata.org/entity/Q19887878'), ('labelsObjectsClasses', 'Wikimedia infobox template')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('labelProperty', 'sport'), ('objectsClasses', 'http://www.wikidata.org/entity/Q212434,http://www.wikidata.org/entity/Q31629'), ('labelsObjectsClasses', 'Olympic sport,type of sport')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('labelProperty', "topic's main category"), ('objectsClasses', 'http://www.wikidata.org/entity/Q4167836'), ('labelsObjectsClasses', 'Wikimedia category')]
4


### Get all the couples (object property - list of the  objects that are connected to "basketball team"  (wd:Q13393265) through that object property)

In [107]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q13393265 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q51747567,http://www.wikidata.org/entity/Q3665646,http://www.wikidata.org/entity/Q5137571'), ('labelsObjects', 'squad,basketball player,basketball coach')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objects', 'http://www.wikidata.org/entity/Q5648512'), ('labelsObjects', 'Template:Infobox basketball club')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('labelProperty', 'sport'), ('objects', 'http://www.wikidata.org/entity/Q5372'), ('labelsObjects', 'basketball')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('labelProperty', "topic's main category"), ('objects', 'http://www.wikidata.org/entity/Q5647277'), ('labelsObjects', 'Category:Basketball teams')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('labelProperty', 'subclass of'), ('objects', 'h

### Get all the objects connected to "basketball team" (wd:Q13393265) with "sport" (wdt:P641)

In [108]:
queryString = """
SELECT DISTINCT ?sport ?labelSport ?sportClass ?labelSportClass
WHERE 
{   
   wd:Q13393265 wdt:P641 ?sport .
   ?sport sc:name ?labelSport ;
          wdt:P31 ?sportClass .
   ?sportClass sc:name ?labelSportClass .   
}

"""
print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('labelSport', 'basketball'), ('sportClass', 'http://www.wikidata.org/entity/Q212434'), ('labelSportClass', 'Olympic sport')]
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('labelSport', 'basketball'), ('sportClass', 'http://www.wikidata.org/entity/Q31629'), ('labelSportClass', 'type of sport')]
2


### Final Query

Final query for this task

In [268]:
queryString = """
SELECT DISTINCT ?sport ?labelSport 
WHERE 
{   
   wd:Q13393265 wdt:P641 ?sport .
   ?sport sc:name ?labelSport .
}

"""
print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q5372'), ('labelSport', 'basketball')]
1


## Task 3
Identify the BGP for basketball team.

### Final Query

Final query for this task

In [278]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q128109 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q13393265'), ('label', 'basketball team')]
1


## Task 4
Return all french basketball teams born before 1930 (the result set must be a list of triples basketball team IRI, label and year of inception).

### Get the value connected with "inception" (wdt:P571) for "Chicago Bulls" (wd:Q128109)

In [120]:
queryString = """
SELECT DISTINCT ?inception
WHERE {   
     wd:Q128109 wdt:P571 ?inception .
}
"""
print("Results")
x=run_query(queryString)

Results
[('inception', '1966-01-01T00:00:00Z')]
1


### Get all data property of "country" (wd:Q6256)

In [121]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q6256 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2347'), ('label', 'YSO ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2452'), ('label', 'GeoNames feature code')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/direct/P

### Get all data property of "country"  (wd:Q6256) that do not contain ID

In [122]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q6256 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P2452'), ('label', 'GeoNames feature code')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P5250'), ('label', 'IAB code')]
[('property', 'http://www.wikidata.org/prop/direct/P7497'), ('label', 'Wolfram Language entity type')]
7


### Get the class of "country" (wd:Q6256)

In [123]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q6256 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q104098715'), ('label', 'territorial entity type')]
1


### Get all object properties of "country" (wd:Q6256)

In [124]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q6256 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P150'), ('label', 'contains administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('property', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P2670'), ('label', 'has parts of the class')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/d

### Get all the classes connected to "country" (wd:Q6256)

In [125]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q6256 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q24239898'), ('label', 'Wikidata property for Wikivoyage listings')]
[('objectClass', 'http://www.wikidata.org/entity/Q24710760'), ('label', 'Wikidata property of quantity type indicating a percentage')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q22964785'), ('label', 'Wikidata property related to events')]
[('objectClass', 'http://www.wikidata.org/entity/Q21286738'), ('label', 'Wikimedia permanent duplicate item')]
[('objectClass', 'http://www.wikidata.org/entity/Q214339'), ('label', 'role')]
[('objectClass', 'http://www.wikidata.org/entity/Q22984026'), ('label', 'Wikidata property related to law and justice')]
[('objectClass', 'http://www.wikidata.org/entity/Q22964372'), ('label', 'Wikidata property related to sport organizations')]
[('objectClass', 'http://www.wikidata.org/entity/Q22965078'), ('label', 'Wikidata property for items ab

### Get all the couples (object property - list of the classes of the objects that are connected to "country" (wd:Q6256) through that object property)

In [126]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q6256 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q15617994'), ('labelsObjectsClasses', 'designation for an administrative territorial entity')]
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objectsClasses', 'http://www.wikidata.org/entity/Q16695773,http://www.wikidata.org/entity/Q13406463,http://www.wikidata.org/entity/Q14204246'), ('labelsObjectsClasses', 'WikiProject,Wikimedia list article,Wikimedia project page')]
[('property', 'http://www.wikidata.org/prop/direct/P5869'), ('labelProperty', 'model item'), ('objectsClasses', 'http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q4209223,http://www.wikidata.org/entity/Q43702,http://www.wikidata.org/entity/Q619610,http://www.wikidata.org/entity/Q63791824,http://www.wikidata.org/entity/Q7270'), ('labelsObjectsClasses', 'sovereign state,Rechtsstaat,f

### Get all the couples (object property - list of the  objects that are connected to "country" (wd:Q6256) through that object property)

In [127]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q6256 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q183366'), ('labelsObjects', 'territory')]
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objects', 'http://www.wikidata.org/entity/Q5460604'), ('labelsObjects', 'Wikipedia:Vital articles')]
[('property', 'http://www.wikidata.org/prop/direct/P5869'), ('labelProperty', 'model item'), ('objects', 'http://www.wikidata.org/entity/Q183'), ('labelsObjects', 'Germany')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objects', 'http://www.wikidata.org/entity/Q5621162'), ('labelsObjects', 'Template:Infobox country')]
[('property', 'http://www.wikidata.org/prop/direct/P2670'), ('labelProperty', 'has parts of the class'), ('objects', 'http://www.wikidata.org/entity/Q1020994'), ('labelsObjects', 'citizen')]
[('property', 'http://

### Get all data property of "United States of America" (wd:Q30)

In [134]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q30 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('label', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1017'), ('label', 'Vatican Library ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1081'), ('label', 'Human Development Index')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1125'), ('label', 'Gini coefficient')]
[('property', 'http://www.wikidata.org/prop/direct/P1188'), ('label', 'Dharma Drum Institute of Liberal Arts place ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1198'), ('label', 'unemployment rate')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki

### Get all data property of "United States of America"  (wd:Q30) that do not contain ID

In [133]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q30 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1081'), ('label', 'Human Development Index')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1125'), ('label', 'Gini coefficient')]
[('property', 'http://www.wikidata.org/prop/direct/P1198'), ('label', 'unemployment rate')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1448'), ('label', 'official name')]
[('property', 'http://www.wikidata.org/prop/direct/P1451'), ('label', 'motto text')]
[('property', 'http://www.wikidata.org/prop/direct/P1549'), ('label', 'demonym')]
[('property', 'http://www.wikidata.org/prop/direct/P1705'), 

### Get all the classes of "United States of America" (wd:Q30)

In [132]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q30 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('class', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('class', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('class', 'http://www.wikidata.org/entity/Q3624078'), ('label', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q512187'), ('label', 'federal republic')]
[('class', 'http://www.wikidata.org/entity/Q5255892'), ('label', 'democratic republic')]
[('class', 'http://www.wikidata.org/entity/Q6256'), ('label', 'country')]
7


### Get all object properties of "United States of America"  (wd:Q30)

In [135]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q30 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P122'), ('label', 'basic form of government')]
[('property', 'http://www.wikidata.org/prop/direct/P1304'), ('label', 'central bank')]
[('property', 'http://www.wikidata.org/prop/direct/P1313'), ('label', 'office held by head of government')]
[('property', 'http://www.wikidata.org/prop/direct/P1325'), ('label', 'external data available at')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('label', 'participant in')]
[('property', 'http://www.wikidata.org/prop/direct/P1365'), ('label', 'replaces')]
[('property', 'http://www.wikidata.org/prop/direct/P138'), ('label', 'named after')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/dir

### Get all the classes connected to "United States of America"  (wd:Q30)

In [130]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q30 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q335214'), ('label', 'endangered language')]
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q12056846'), ('label', 'national seal')]
[('objectClass', 'http://www.wikidata.org/entity/Q63100611'), ('label', 'category 5 hurricane')]
[('objectClass', 'http://www.wikidata.org/entity/Q1006311'), ('label', 'war of national liberation')]
[('objectClass', 'http://www.wikidata.org/entity/Q114380'), ('label', 'financial crisis')]
[('objectClass', 'http://www.wikidata.org/entity/Q1006644'), ('label', 'federal government')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q4688003'), ('label', 'aerial bombing of a city')]
[('objectClass', 'http://www.wikidata.org/entity/Q132821'), ('label', 'murder')]
[('objectClass', 'http://www.wikidata.org/entity/Q1328899'), ('label'

### Get all the couples (object property - list of the classes of the objects that are connected to "United States of America"  (wd:Q30) through that object property)

In [129]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q30 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objectsClasses', 'http://www.wikidata.org/entity/Q11862829,http://www.wikidata.org/entity/Q4671286,http://www.wikidata.org/entity/Q1047113,http://www.wikidata.org/entity/Q17544377'), ('labelsObjectsClasses', 'academic discipline,academic major,specialty,history of a country or state')]
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objectsClasses', 'http://www.wikidata.org/entity/Q26884324'), ('labelsObjectsClasses', 'Wikimedia outline article')]
[('property', 'http://www.wikidata.org/prop/direct/P2238'), ('labelProperty', 'official symbol'), ('objectsClasses', 'http://www.wikidata.org/entity/Q16521'), ('labelsObjectsClasses', 'taxon')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5107,http://www.wikidata.org/entity/Q855697'), ('l

### Get all the couples (object property - list of the  objects that are connected to "United States of America"  (wd:Q30) through that object property)

In [128]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q30 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objects', 'http://www.wikidata.org/entity/Q131110'), ('labelsObjects', 'history of the United States')]
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objects', 'http://www.wikidata.org/entity/Q7112786'), ('labelsObjects', 'outline of the United States')]
[('property', 'http://www.wikidata.org/prop/direct/P2238'), ('labelProperty', 'official symbol'), ('objects', 'http://www.wikidata.org/entity/Q127216'), ('labelsObjects', 'bald eagle')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objects', 'http://www.wikidata.org/entity/Q49'), ('labelsObjects', 'North America')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('labelProperty', 'participant in'), ('objects', 'http://www.wikidata.org/entity/Q418151,http://www.wikidata.org/entity/Q301487,http://www.wikidata.org/entity

### Get the values of "official name" (wdt:P1448) of "United States of America" (wd:Q30)

In [158]:
queryString = """
SELECT DISTINCT ?officialName
WHERE {   
    wd:Q30 wdt:P1448 ?officialName .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('officialName', 'United States')]
[('officialName', 'the United States of America')]
2


### Get first 50 objects connected with "nationality" (wdt:P27) such that the their labels contains "France"

In [290]:
queryString = """
SELECT DISTINCT ?obj ?labelObj
WHERE {   
    ?sbj wdt:P27 ?obj .   
    ?sbj sc:name ?labelSbj .
    ?obj sc:name ?labelObj .
    FILTER (CONTAINS(?labelObj, "France"))
} LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q170604'), ('labelObj', 'New France')]
[('obj', 'http://www.wikidata.org/entity/Q69323'), ('labelObj', 'Kingdom of France')]
[('obj', 'http://www.wikidata.org/entity/Q142'), ('labelObj', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q70972'), ('labelObj', 'Kingdom of France')]
[('obj', 'http://www.wikidata.org/entity/Q69808'), ('labelObj', 'Vichy France')]
[('obj', 'http://www.wikidata.org/entity/Q4055129'), ('labelObj', 'absolute monarchy in France')]
[('obj', 'http://www.wikidata.org/entity/Q237131'), ('labelObj', 'France in the Middle Ages')]
[('obj', 'http://www.wikidata.org/entity/Q18606521'), ('labelObj', 'Francesa')]
[('obj', 'http://www.wikidata.org/entity/Q5478259'), ('labelObj', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q1573912'), ('labelObj', 'Kingdom of France')]
10


### Get all data property of "France" (wd:Q142)

In [283]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q142 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1005'), ('label', 'Portuguese National Library ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1015'), ('label', 'NORAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1081'), ('label', 'Human Development Index')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1120'), ('label', 'number of deaths')]
[('property', 'http://www.wikidata.org/prop/direct/P1125'), ('label', 'Gini coefficient')]
[('property', 'http://www.wikidata.org/prop/direct/P1198'), ('label', 'unemployment rate')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.o

### Get all data property of "France"  (wd:Q142) that do not contain ID

In [284]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q142 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1081'), ('label', 'Human Development Index')]
[('property', 'http://www.wikidata.org/prop/direct/P1082'), ('label', 'population')]
[('property', 'http://www.wikidata.org/prop/direct/P1120'), ('label', 'number of deaths')]
[('property', 'http://www.wikidata.org/prop/direct/P1125'), ('label', 'Gini coefficient')]
[('property', 'http://www.wikidata.org/prop/direct/P1198'), ('label', 'unemployment rate')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1549'), ('label', 'demonym')]
[('property', 'http://www.wikidata.org/prop/direct/P2046'), ('label', 'area')]
[('property', 'http://www.wikidata.org/prop/direct/P2250'), ('l

### Get the class of "France" (wd:Q142)

In [285]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q142 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3624078'), ('label', 'sovereign state')]
[('class', 'http://www.wikidata.org/entity/Q51576574'), ('label', 'Mediterranean country')]
[('class', 'http://www.wikidata.org/entity/Q6256'), ('label', 'country')]
[('class', 'http://www.wikidata.org/entity/Q7270'), ('label', 'republic')]
4


### Get all object properties of "France"  (wd:Q142)

In [286]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q142 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P122'), ('label', 'basic form of government')]
[('property', 'http://www.wikidata.org/prop/direct/P1313'), ('label', 'office held by head of government')]
[('property', 'http://www.wikidata.org/prop/direct/P1325'), ('label', 'external data available at')]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('label', 'participant in')]
[('property', 'http://www.wikidata.org/prop/direct/P1365'), ('label', 'replaces')]
[('property', 'http://www.wikidata.org/prop/direct/P138'), ('label', 'named after')]
[('property', 'http://www.wikidata.org/prop/direct/P1464'), ('label', 'category for people born here')]
[('property', 'http://www.wikidata.org/prop/direct/P1465'), ('label', 'category for people who died here')]
[('property', 'htt

### Get all the classes connected to "France"  (wd:Q142)

In [287]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q142 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q12056846'), ('label', 'national seal')]
[('objectClass', 'http://www.wikidata.org/entity/Q1345691'), ('label', 'international financial institution')]
[('objectClass', 'http://www.wikidata.org/entity/Q32766'), ('label', 'constitutional court')]
[('objectClass', 'http://www.wikidata.org/entity/Q10210'), ('label', 'white wine')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q1328899'), ('label', 'standards organization')]
[('objectClass', 'http://www.wikidata.org/entity/Q15925165'), ('label', 'specialized agency of the United Nations')]
[('objectClass', 'http://www.wikidata.org/entity/Q35798'), ('label', 'executive branch')]
[('objectClass', 'http://www.wikidata.org/entity/Q7755'), ('label', 'constitution')]
[('objectClass', 'http://www.wikidata.org/entity/

### Get all the couples (object property - list of the classes of the objects that are connected to "France"  (wd:Q142) through that object property)

In [288]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q142 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objectsClasses', 'http://www.wikidata.org/entity/Q11862829'), ('labelsObjectsClasses', 'academic discipline')]
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objectsClasses', 'http://www.wikidata.org/entity/Q15851373,http://www.wikidata.org/entity/Q26884324'), ('labelsObjectsClasses', 'crosslink page,Wikimedia outline article')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objectsClasses', 'http://www.wikidata.org/entity/Q82794,http://www.wikidata.org/entity/Q153026'), ('labelsObjectsClasses', 'geographic region,Euroregion')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('labelProperty', 'participant in'), ('objectsClasses', 'http://www.wikidata.org/entity/Q198'), ('labelsObjectsClasses', 'war')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelPro

### Get all the couples (object property - list of the  objects that are connected to "France"  (wd:Q142) through that object property)

In [289]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q142 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objects', 'http://www.wikidata.org/entity/Q7778'), ('labelsObjects', 'history of France')]
[('property', 'http://www.wikidata.org/prop/direct/P5125'), ('labelProperty', 'Wikimedia outline'), ('objects', 'http://www.wikidata.org/entity/Q4261422'), ('labelsObjects', 'outline of France')]
[('property', 'http://www.wikidata.org/prop/direct/P361'), ('labelProperty', 'part of'), ('objects', 'http://www.wikidata.org/entity/Q27496,http://www.wikidata.org/entity/Q1849394'), ('labelsObjects', 'Western Europe,Pyrenees–Mediterranean Euroregion')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('labelProperty', 'participant in'), ('objects', 'http://www.wikidata.org/entity/Q766437'), ('labelsObjects', 'Rif War')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q1050483'), ('labelsObjects', 'Loi

### Get the first 50 "basketball team" (wd:Q13393265) that have "country" (wdt:P17) equal to "France" (wd:Q142)

In [291]:
queryString = """
SELECT DISTINCT ?team ?labelTeam
WHERE 
{   
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P17 wd:Q142 ;
          sc:name ?labelTeam .
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q795707'), ('labelTeam', 'BCM Gravelines')]
[('team', 'http://www.wikidata.org/entity/Q1075934'), ('labelTeam', 'Cholet Basket')]
[('team', 'http://www.wikidata.org/entity/Q2892731'), ('labelTeam', 'Paris Université Club')]
[('team', 'http://www.wikidata.org/entity/Q404421'), ('labelTeam', 'STB Le Havre')]
[('team', 'http://www.wikidata.org/entity/Q2887227'), ('labelTeam', 'Basket Landes')]
[('team', 'http://www.wikidata.org/entity/Q40063'), ('labelTeam', 'Le Mans Sarthe Basket')]
[('team', 'http://www.wikidata.org/entity/Q2868091'), ('labelTeam', 'AS Monaco Basket')]
[('team', 'http://www.wikidata.org/entity/Q2352349'), ('labelTeam', 'JA Vichy')]
[('team', 'http://www.wikidata.org/entity/Q3178083'), ('labelTeam', 'JSA Bordeaux Basket')]
[('team', 'http://www.wikidata.org/entity/Q3532914'), ('labelTeam', 'Toulouse Métropole Basket')]
[('team', 'http://www.wikidata.org/entity/Q3549999'), ('labelTeam', 'LDLC ASVEL féminin')]
[('team', 'ht

### Final Query

Final query for this task

In [297]:
queryString = """
SELECT DISTINCT ?team ?labelTeam ?year
WHERE {   
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P17 wd:Q142 ;
          wdt:P571 ?inception ;
          sc:name ?labelTeam .
    BIND(YEAR(?inception) AS ?year) .
    FILTER (?year < 1930)
          
}
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q2892731'), ('labelTeam', 'Paris Université Club'), ('year', '1906')]
[('team', 'http://www.wikidata.org/entity/Q404421'), ('labelTeam', 'STB Le Havre'), ('year', '1924')]
[('team', 'http://www.wikidata.org/entity/Q2868091'), ('labelTeam', 'AS Monaco Basket'), ('year', '1924')]
[('team', 'http://www.wikidata.org/entity/Q1630650'), ('labelTeam', 'JL Bourg-en-Bresse'), ('year', '1910')]
[('team', 'http://www.wikidata.org/entity/Q2868120'), ('labelTeam', 'Union Tours Basket Métropole'), ('year', '1925')]
[('team', 'http://www.wikidata.org/entity/Q2931972'), ('labelTeam', 'Basket CRO Lyon'), ('year', '1927')]
[('team', 'http://www.wikidata.org/entity/Q2887216'), ('labelTeam', 'BC Orchies'), ('year', '1922')]
[('team', 'http://www.wikidata.org/entity/Q3592481'), ('labelTeam', 'Étoile de Charleville-Mézières'), ('year', '1921')]
[('team', 'http://www.wikidata.org/entity/Q638938'), ('labelTeam', 'Paris Basket Racing'), ('year', '1922')]
[('tea

## Task 5
Return all the NBA season (the result set must be a list of couples NBA season IRI, label)

### Get all data property of "National Basketball Association" (wd:Q155223)

In [180]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q155223 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1132'), ('label', 'number of participants')]
[('property', 'http://www.wikidata.org/prop/direct/P1273'), ('label', 'CANTIC ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P2013'), ('label', 'Facebook ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2088'), ('label', 'Crunchbase organization ID')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P214'), ('label', 'VIAF ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2257'), ('label', 'event interval')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('p

### Get all data property of "National Basketball Association"  (wd:Q155223) that do not contain ID

In [181]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q155223 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1132'), ('label', 'number of participants')]
[('property', 'http://www.wikidata.org/prop/direct/P2002'), ('label', 'Twitter username')]
[('property', 'http://www.wikidata.org/prop/direct/P2003'), ('label', 'Instagram username')]
[('property', 'http://www.wikidata.org/prop/direct/P213'), ('label', 'ISNI')]
[('property', 'http://www.wikidata.org/prop/direct/P2257'), ('label', 'event interval')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P3984'), ('label', 'subreddit')]
[('property', 'http://www.wikidata.org/prop/direct/P4013'), ('label', 'Giphy username')]
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
[('property', 'http://www.wikidata.org/prop/direct/P7085'), ('label', 'TikTok username')]
[('property', 'http://www.wikidata.org/prop/direct/P8687'), ('label', 'social media followers')]
11


### Get the class of "National Basketball Association" (wd:Q155223)

In [182]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q155223 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q18536323'), ('label', 'basketball league')]
[('class', 'http://www.wikidata.org/entity/Q15986354'), ('label', 'basketball association')]
[('class', 'http://www.wikidata.org/entity/Q15991290'), ('label', 'professional sports league')]
[('class', 'http://www.wikidata.org/entity/Q7094076'), ('label', 'online database')]
4


### Get all object properties of "National Basketball Association"  (wd:Q155223)

In [183]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q155223 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P6118'), ('label', 'season ends')]
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P1365'), ('label', 'replaces')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P154'), ('label', 'logo image')]
[('property', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1830'), ('label', 'owner of')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('label', 'competition class')]
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('label', 'history of topic')]
[('property', 'http://www.wikidata.org/prop/di

### Get all the classes connected to "National Basketball Association"  (wd:Q155223)

In [184]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q155223 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q22936940'), ('label', 'competition class')]
[('objectClass', 'http://www.wikidata.org/entity/Q2001305'), ('label', 'television channel')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q15986354'), ('label', 'basketball association')]
[('objectClass', 'http://www.wikidata.org/entity/Q11266439'), ('label', 'Wikimedia template')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406554'), ('label', 'sports competition')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectClass', 'http://www.wikidata.org/entity/Q17524420'), ('label', 'aspect of history')]
[('objectClass', 'http://www.wikidata.org/entity/Q18608583'), ('label', 'recurring sporting event')]
[('objectClass', 'http://www.

### Get all the couples (object property - list of the classes of the objects that are connected to "National Basketball Association"  (wd:Q155223) through that object property)

In [185]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q155223 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2184'), ('labelProperty', 'history of topic'), ('objectsClasses', 'http://www.wikidata.org/entity/Q17524420'), ('labelsObjectsClasses', 'aspect of history')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q13406554,http://www.wikidata.org/entity/Q13406554'), ('labelsObjectsClasses', 'sports competition,sports competition')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objectsClasses', 'http://www.wikidata.org/entity/Q11266439'), ('labelsObjectsClasses', 'Wikimedia template')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objectsClasses', 'http://www.wikidata.org/entity/Q1489259,http://www.wikidata.org/entity/Q99541706,http://www.wikidata.org/entity/Q1520223,http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q362

### Get all the couples (object property - list of the  objects that are connected to "National Basketball Association"  (wd:Q155223) through that object property)

In [186]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q155223 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q238240,http://www.wikidata.org/entity/Q318389'), ('labelsObjects', 'Eastern Conference,Western Conference')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objects', 'http://www.wikidata.org/entity/Q10950388'), ('labelsObjects', 'Template:NBA seasons')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objects', 'http://www.wikidata.org/entity/Q16,http://www.wikidata.org/entity/Q30'), ('labelsObjects', 'Canada,United States of America')]
[('property', 'http://www.wikidata.org/prop/direct/P1830'), ('labelProperty', 'owner of'), ('objects', 'http://www.wikidata.org/entity/Q2455323,http://www.wikidata.org/entity/Q6952699,http://www.wikidata.org/entity/Q1470803'), ('labelsObjects', 'NBA TV,NBA Store,Three-Point Contest')]
[('property', 'http://www.wiki

### Get the first 25 subjects connected to "basketball" (wd:Q5372) with "sport" (wdt:P641)

In [190]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj 
WHERE {   
   ?sbj wdt:P641 wd:Q5372 ;
        sc:name ?labelSbj .
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q16191059'), ('labelSbj', "2013–14 Atlantic Coast Conference men's basketball season")]
[('sbj', 'http://www.wikidata.org/entity/Q16192074'), ('labelSbj', "2013–14 Southeastern Conference men's basketball season")]
[('sbj', 'http://www.wikidata.org/entity/Q19863393'), ('labelSbj', "2008–09 Atlantic 10 Conference men's basketball season")]
[('sbj', 'http://www.wikidata.org/entity/Q2348425'), ('labelSbj', 'sports in Sweden')]
[('sbj', 'http://www.wikidata.org/entity/Q2811918'), ('labelSbj', '1947 in basketball')]
[('sbj', 'http://www.wikidata.org/entity/Q2813083'), ('labelSbj', '1991 in basketball')]
[('sbj', 'http://www.wikidata.org/entity/Q2813265'), ('labelSbj', '1996 in basketball')]
[('sbj', 'http://www.wikidata.org/entity/Q2813922'), ('labelSbj', '2001 in basketball')]
[('sbj', 'http://www.wikidata.org/entity/Q2855716'), ('labelSbj', '2012–13 ACB season')]
[('sbj', 'http://www.wikidata.org/entity/Q2948730'), ('labelSbj', "2010 NCAA M

### Get the first 25 subjects connected to "basketball" (wd:Q5372) with "sport" (wdt:P641) such that their label contains "NBA season"

In [194]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj 
WHERE {   
   ?sbj wdt:P641 wd:Q5372 ;
        sc:name ?labelSbj .
   FILTER CONTAINS(?labelSbj, "NBA season")
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q600389'), ('labelSbj', '1990–91 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q2264531'), ('labelSbj', '1967–68 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q241834'), ('labelSbj', '1989–90 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q17186677'), ('labelSbj', 'Template:2014–15 NBA season by team')]
[('sbj', 'http://www.wikidata.org/entity/Q602553'), ('labelSbj', '1969–70 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q2294127'), ('labelSbj', '1971–72 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q113619'), ('labelSbj', '2012–13 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q16191100'), ('labelSbj', '2014-15 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q19862716'), ('labelSbj', '2015–16 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q631240'), ('labelSbj', '1979–80 NBA season')]
[('sbj', 'http://www.wikidata.org/entity/Q3255071'), ('labelSbj', 'list of 2012–13 NBA 

### Get all data property of "2018–19 NBA season" (wd:Q52702755)

In [201]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52702755 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1132'), ('label', 'number of participants')]
[('property', 'http://www.wikidata.org/prop/direct/P1350'), ('label', 'number of matches played/races/starts')]
[('property', 'http://www.wikidata.org/prop/direct/P2671'), ('label', 'Google Knowledge Graph ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3553'), ('label', 'Zhihu topic ID')]
[('property', 'http://www.wikidata.org/prop/direct/P393'), ('label', 'edition number')]
[('property', 'http://www.wikidata.org/prop/direct/P580'), ('label', 'start time')]
[('property', 'http://www.wikidata.org/prop/direct/P582'), ('label', 'end time')]
7


### Get all data property of "2018–19 NBA season"  (wd:Q52702755) that do not contain ID

In [200]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52702755 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1132'), ('label', 'number of participants')]
[('property', 'http://www.wikidata.org/prop/direct/P1350'), ('label', 'number of matches played/races/starts')]
[('property', 'http://www.wikidata.org/prop/direct/P393'), ('label', 'edition number')]
[('property', 'http://www.wikidata.org/prop/direct/P580'), ('label', 'start time')]
[('property', 'http://www.wikidata.org/prop/direct/P582'), ('label', 'end time')]
5


### Get the class of "2018–19 NBA season" (wd:Q52702755)

In [199]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q52702755 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q27020041'), ('label', 'sports season')]
1


### Get all object properties of "2018–19 NBA season"  (wd:Q52702755)

In [198]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q52702755 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('label', 'country')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('label', 'competition class')]
[('property', 'http://www.wikidata.org/prop/direct/P2348'), ('label', 'time period')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P3450'), ('label', 'sports season of league or competition')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P664'), ('label', 'organizer')]
[('property', 'http://www.wikidata.org/prop/direct/P856'), ('label', 'official website')]
[('property', 'http://www.wikidata.org/prop/direct/P910'), ('label', "topic's main category")]
[('property', 'http://www.wikidata.org/prop/direct/P647'), ('label', 'drafted by')]
11


### Get all the classes connected to "2018–19 NBA season"  (wd:Q52702755)

In [197]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q52702755 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q22936940'), ('label', 'competition class')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q18536323'), ('label', 'basketball league')]
[('objectClass', 'http://www.wikidata.org/entity/Q622521'), ('label', 'National Basketball Association Draft')]
[('objectClass', 'http://www.wikidata.org/entity/Q15986354'), ('label', 'basketball association')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectClass', 'http://www.wikidata.org/entity/Q15991290'), ('label', 'professional sports league')]
[('objectClass', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
[('objectClass', 'http:/

### Get all the couples (object property - list of the classes of the objects that are connected to "2018–19 NBA season"  (wd:Q52702755) through that object property)

In [196]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q52702755 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P647'), ('labelProperty', 'drafted by'), ('objectsClasses', 'http://www.wikidata.org/entity/Q622521'), ('labelsObjectsClasses', 'National Basketball Association Draft')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objectsClasses', 'http://www.wikidata.org/entity/Q1489259,http://www.wikidata.org/entity/Q99541706,http://www.wikidata.org/entity/Q1520223,http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q3624078,http://www.wikidata.org/entity/Q512187,http://www.wikidata.org/entity/Q5255892,http://www.wikidata.org/entity/Q6256,http://www.wikidata.org/entity/Q6256'), ('labelsObjectsClasses', 'superpower,historical unrecognized state,constitutional republic,sovereign state,sovereign state,federal republic,democratic republic,country,country')]
[('property', 'http://www.wikidata.org/prop/direct/P3450'), ('labelProperty', 'sports season of league or competition'), ('obje

### Get all the couples (object property - list of the  objects that are connected to "2018–19 NBA season"  (wd:Q52702755) through that object property)

In [195]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q52702755 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P647'), ('labelProperty', 'drafted by'), ('objects', 'http://www.wikidata.org/entity/Q47329390'), ('labelsObjects', '2018 NBA draft')]
[('property', 'http://www.wikidata.org/prop/direct/P17'), ('labelProperty', 'country'), ('objects', 'http://www.wikidata.org/entity/Q16,http://www.wikidata.org/entity/Q30'), ('labelsObjects', 'Canada,United States of America')]
[('property', 'http://www.wikidata.org/prop/direct/P3450'), ('labelProperty', 'sports season of league or competition'), ('objects', 'http://www.wikidata.org/entity/Q155223'), ('labelsObjects', 'National Basketball Association')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('labelProperty', 'sport'), ('objects', 'http://www.wikidata.org/entity/Q5372'), ('labelsObjects', 'basketball')]
[('property', 'http://www.wikidata.org/prop/direct/P664'), ('labelProperty', 'organizer'), ('objects', 'http://www.wikidata.org/entity/Q155223'), ('labelsObjects', 'National B

### Get all "sports season" (wd:Q27020041) that have "sports season of league or competition" (wdt:P3450) "National Basketball Association" (wd:Q155223) without "Summer" and "BAA" in the label

In [213]:
queryString = """
SELECT DISTINCT ?season ?labelSeason 
WHERE 
{   
    ?season wdt:P31 wd:Q27020041 ;
            wdt:P3450 wd:Q155223 ;
            sc:name ?labelSeason .
    FILTER (!REGEX (?labelSeason, "Summer") && !REGEX(?labelSeason, "BAA"))
            
}ORDER BY DESC (?labelSeason)
"""
print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q106495853'), ('labelSeason', '2021–22 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q96083208'), ('labelSeason', '2020–21 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q63637259'), ('labelSeason', '2019–20 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q52702755'), ('labelSeason', '2018–19 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q29098111'), ('labelSeason', '2017–18 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q23939796'), ('labelSeason', '2016–17 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q19862716'), ('labelSeason', '2015–16 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q16191100'), ('labelSeason', '2014-15 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q11144228'), ('labelSeason', '2013–14 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q113619'), ('labelSeason', '2012–13 NBA season')]
[('season', 'http://www.wikidata.

### Final Query

Final query for this task

In [214]:
queryString = """
SELECT DISTINCT ?season ?labelSeason 
WHERE 
{   
    ?season wdt:P31 wd:Q27020041 ;
            wdt:P3450 wd:Q155223 ;
            sc:name ?labelSeason .
    FILTER (!REGEX (?labelSeason, "Summer") && !REGEX(?labelSeason, "BAA"))            
}
"""
print("Results")
x=run_query(queryString)

Results
[('season', 'http://www.wikidata.org/entity/Q600389'), ('labelSeason', '1990–91 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2264531'), ('labelSeason', '1967–68 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q241834'), ('labelSeason', '1989–90 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q602553'), ('labelSeason', '1969–70 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q2294127'), ('labelSeason', '1971–72 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q113619'), ('labelSeason', '2012–13 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q16191100'), ('labelSeason', '2014-15 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q19862716'), ('labelSeason', '2015–16 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q631240'), ('labelSeason', '1979–80 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q1142320'), ('labelSeason', '2005–06 NBA season')]
[('season', 'http://www.wikidata.org/entity/Q

## Task 6
Consider only the basketball teams which Michael Jordan and Kobe Bryant played. Return the number of NBA seasons won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team even if they never won a season).

### Get all "Basketball Team" (wd:Q155223) in which "Micheal Jordan" (wd:Q41421) and "Kobe Bryant" (wd:Q25369) were "member of sports team" (wdt:P54)

In [429]:
queryString = """
SELECT DISTINCT ?team ?labelTeam
WHERE 
{   
    ?player wdt:P54 ?team .
    ?team wdt:P118 wd:Q155223 .
    ?team sc:name ?labelTeam . 
    VALUES (?player)
    { 
        (wd:Q41421) (wd:Q25369)
    }    
} LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q121783'), ('labelTeam', 'Los Angeles Lakers')]
[('team', 'http://www.wikidata.org/entity/Q128109'), ('labelTeam', 'Chicago Bulls')]
[('team', 'http://www.wikidata.org/entity/Q169165'), ('labelTeam', 'Washington Wizards')]
3


### Get the number of NBA Seasons won (rings) by "Chicago Bulls" (wd:Q128109)

In [221]:
queryString = """
SELECT DISTINCT (COUNT(?season) AS ?totalSeasonsWon)
WHERE 
{   
    ?season wdt:P31 wd:Q27020041 ;
            wdt:P3450 wd:Q155223 ;
            wdt:P1346 wd:Q128109 ; #winner
            sc:name ?labelSeason .
    FILTER (!REGEX (?labelSeason, "Summer") && !REGEX(?labelSeason, "BAA"))            
}
"""
print("Results")
x=run_query(queryString)

Results
[('totalSeasonsWon', '6')]
1


### Final Query

Final query for this task

In [430]:
queryString = """
SELECT DISTINCT ?winner ?labelWinner (COUNT(?season) AS ?totalSeasonsWon)
WHERE 
{   
    ?season wdt:P31 wd:Q27020041 ;
            wdt:P3450 wd:Q155223 ;
            wdt:P1346 ?winner ; #winner
            sc:name ?labelSeason .
    ?winner sc:name ?labelWinner .
    FILTER (!REGEX (?labelSeason, "Summer") && !REGEX(?labelSeason, "BAA")) .               
    FILTER (?winner = ?team && ?labelWinner = ?labelTeam) .
    {
        SELECT DISTINCT ?team ?labelTeam
        WHERE 
        {   
            ?player wdt:P54 ?team .
            ?team wdt:P118 wd:Q155223 .
            ?team sc:name ?labelTeam . 
            VALUES (?player)
            { 
                (wd:Q41421) (wd:Q25369)
            }    
        } LIMIT 50  
    }       
}
"""
print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q128109'), ('labelWinner', 'Chicago Bulls'), ('totalSeasonsWon', '6')]
[('winner', 'http://www.wikidata.org/entity/Q121783'), ('labelWinner', 'Los Angeles Lakers'), ('totalSeasonsWon', '16')]
[('winner', 'http://www.wikidata.org/entity/Q169165'), ('labelWinner', 'Washington Wizards'), ('totalSeasonsWon', '1')]
3


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, label and #players)

### Get the "continent" (wdt:P30) of "France" (wd:Q142)

In [299]:
queryString = """
SELECT DISTINCT ?continent ?labelContinent
WHERE 
{   
    wd:Q142 wdt:P30 ?continent .
    ?continent sc:name ?labelContinent .
}
"""
print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('labelContinent', 'Europe')]
1


### Get all the "basketball team" (wd:Q13393265) that have "league" (wdt:P118) "National Basketball Association" (wd:Q155223)

In [301]:
queryString = """
SELECT DISTINCT ?team ?labelTeam
WHERE 
{   
    ?team wdt:P31 wd:Q13393265 ;
          wdt:P118 wd:Q155223 ;
          sc:name ?labelTeam .
}
"""
print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q131364'), ('labelTeam', 'New York Knicks')]
[('team', 'http://www.wikidata.org/entity/Q162954'), ('labelTeam', 'Denver Nuggets')]
[('team', 'http://www.wikidata.org/entity/Q167253'), ('labelTeam', 'Portland Trail Blazers')]
[('team', 'http://www.wikidata.org/entity/Q180950'), ('labelTeam', 'Oklahoma City Thunder')]
[('team', 'http://www.wikidata.org/entity/Q976396'), ('labelTeam', 'Los Angeles Clippers')]
[('team', 'http://www.wikidata.org/entity/Q166105'), ('labelTeam', 'Sacramento Kings')]
[('team', 'http://www.wikidata.org/entity/Q170649'), ('labelTeam', 'Utah Jazz')]
[('team', 'http://www.wikidata.org/entity/Q138089'), ('labelTeam', 'Philadelphia 76ers')]
[('team', 'http://www.wikidata.org/entity/Q161345'), ('labelTeam', 'Houston Rockets')]
[('team', 'http://www.wikidata.org/entity/Q164177'), ('labelTeam', 'Phoenix Suns')]
[('team', 'http://www.wikidata.org/entity/Q172339'), ('labelTeam', 'New Orleans Pelicans')]
[('team', 'http://

### Get first 50 players that have "league" (wdt:P118) "National Basketball Association" (wd:Q155223)

In [306]:
queryString = """
SELECT DISTINCT ?player ?labelPlayer
WHERE 
{   
    ?player wdt:P31 wd:Q5 ;
            wdt:P118 wd:Q155223 ;
            sc:name ?labelPlayer .
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q15908865'), ('labelPlayer', 'Jordan Adams')]
[('player', 'http://www.wikidata.org/entity/Q16227014'), ('labelPlayer', 'Tyler Ennis')]
[('player', 'http://www.wikidata.org/entity/Q16233299'), ('labelPlayer', 'Sean Kilpatrick')]
[('player', 'http://www.wikidata.org/entity/Q16236541'), ('labelPlayer', 'Sindarius Thornwell')]
[('player', 'http://www.wikidata.org/entity/Q17623132'), ('labelPlayer', 'C. J. Williams')]
[('player', 'http://www.wikidata.org/entity/Q498348'), ('labelPlayer', 'Jimmy Butler')]
[('player', 'http://www.wikidata.org/entity/Q23302736'), ('labelPlayer', 'Joel Bolomboy')]
[('player', 'http://www.wikidata.org/entity/Q29166745'), ('labelPlayer', 'P. J. Dozier')]
[('player', 'http://www.wikidata.org/entity/Q127125'), ('labelPlayer', 'Mark Price')]
[('player', 'http://www.wikidata.org/entity/Q2351523'), ('labelPlayer', 'Mitchell Wiggins')]
[('player', 'http://www.wikidata.org/entity/Q442623'), ('labelPlayer', 'Al Jefferso

### Final Query

Final query for this task

In [316]:
queryString = """
SELECT DISTINCT ?continentPlayer ?labelContinentPlayer (COUNT (DISTINCT (?player)) AS ?numberOfPlayers)
WHERE 
{   
    # All Nba Players and their continent
    ?player wdt:P31 wd:Q5 ;
            wdt:P118 wd:Q155223 ;
            wdt:P27 ?countryPlayer ;
            sc:name ?labelPlayer .
            
    ?countryPlayer wdt:P30 ?continentPlayer .
    
    ?continentPlayer sc:name ?labelContinentPlayer .
}GROUP BY ?continentPlayer ?labelContinentPlayer
"""
print("Results")
x=run_query(queryString)

Results
[('continentPlayer', 'http://www.wikidata.org/entity/Q46'), ('labelContinentPlayer', 'Europe'), ('numberOfPlayers', '339')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q538'), ('labelContinentPlayer', 'Insular Oceania'), ('numberOfPlayers', '3582')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q15'), ('labelContinentPlayer', 'Africa'), ('numberOfPlayers', '70')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q48'), ('labelContinentPlayer', 'Asia'), ('numberOfPlayers', '82')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q5401'), ('labelContinentPlayer', 'Eurasia'), ('numberOfPlayers', '22')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q18'), ('labelContinentPlayer', 'South America'), ('numberOfPlayers', '37')]
[('continentPlayer', 'http://www.wikidata.org/entity/Q49'), ('labelContinentPlayer', 'North America'), ('numberOfPlayers', '3631')]
7


## Task 8
Identify the BGP for the NBA Defensive Player of the Year Award (see https://en.wikipedia.org/wiki/NBA_Defensive_Player_of_the_Year_Award)

### Final Query

Final query for this task

In [231]:
queryString = """
SELECT DISTINCT ?award ?labelAward
WHERE 
{   
    wd:Q41421 wdt:P166 ?award .
    ?award sc:name ?labelAward .
}
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1061233'), ('labelAward', "L'Équipe Champion of Champions")]
[('award', 'http://www.wikidata.org/entity/Q17144'), ('labelAward', 'Presidential Medal of Freedom')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('labelAward', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q137003'), ('labelAward', '50 Greatest Players in NBA History')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('labelAward', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('labelAward', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1641970'), ('labelAward', 'John R. Wooden Award')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('labelAward', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('labelAward', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q2404133'), ('la

## Task 9
Consider the players who won the NBA Defensive Player of the Year Award. Return for these players the number of NBA Defensive Player of the Year Award won in their career (the result set must be triples of player IRI, label and #victories)

### Get all subjects that are connected with "award received" (wdt:P166) to "NBA Defensive Player of the Year Award" (wd:Q845884)

In [428]:
queryString = """
SELECT DISTINCT ?player ?labelPlayer
WHERE 
{   
    ?player wdt:P166 wd:Q845884 ;
            sc:name ?labelPlayer .
}
"""
print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q177928'), ('labelPlayer', 'Sidney Moncrief')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('labelPlayer', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('labelPlayer', 'Michael Cooper')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('labelPlayer', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('labelPlayer', 'Michael Jordan')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('labelPlayer', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('labelPlayer', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('labelPlayer', 'Marcus Camby')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('labelPlayer', 'Metta Sandiford-Artest')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('labelPlayer', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('labelPlayer', 'Joakim Noah')]

### Get all data property of "NBA Defensive Player of the Year Award" (wd:Q845884)

In [234]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q845884 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('label', 'Freebase ID')]
2


### Get all data property of "NBA Defensive Player of the Year Award"  (wd:Q845884) that do not contain ID

In [235]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q845884 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P571'), ('label', 'inception')]
1


### Get the class of "NBA Defensive Player of the Year Award" (wd:Q845884)

In [236]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q845884 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q618779'), ('label', 'award')]
[('class', 'http://www.wikidata.org/entity/Q38033430'), ('label', 'class of award')]
[('class', 'http://www.wikidata.org/entity/Q15623926'), ('label', 'Wikimedia set index article')]
3


### Get all object properties of "NBA Defensive Player of the Year Award"  (wd:Q845884)

In [237]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q845884 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
3


### Get all the classes connected to "NBA Defensive Player of the Year Award"  (wd:Q845884)

In [238]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q845884 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q5'), ('label', 'human')]
[('objectClass', 'http://www.wikidata.org/entity/Q24017414'), ('label', 'second-order class')]
4


### Get all the couples (object property - list of the classes of the objects that are connected to "NBA Defensive Player of the Year Award"  (wd:Q845884) through that object property)

In [240]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q845884 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('labelProperty', 'sport'), ('objectsClasses', 'http://www.wikidata.org/entity/Q212434,http://www.wikidata.org/entity/Q31629'), ('labelsObjectsClasses', 'Olympic sport,type of sport')]
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('labelProperty', 'winner'), ('objectsClasses', 'http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/entity/Q5,http://www.wikidata.org/ent

### Get all the couples (object property - list of the  objects that are connected to "NBA Defensive Player of the Year Award"  (wd:Q845884) through that object property)

In [241]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q845884 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('labelProperty', 'sport'), ('objects', 'http://www.wikidata.org/entity/Q5372'), ('labelsObjects', 'basketball')]
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('labelProperty', 'winner'), ('objects', 'http://www.wikidata.org/entity/Q177928,http://www.wikidata.org/entity/Q310968,http://www.wikidata.org/entity/Q971538,http://www.wikidata.org/entity/Q189240,http://www.wikidata.org/entity/Q41421,http://www.wikidata.org/entity/Q273256,http://www.wikidata.org/entity/Q211876,http://www.wikidata.org/entity/Q361134,http://www.wikidata.org/entity/Q295322,http://www.wikidata.org/entity/Q926248,http://www.wikidata.org/entity/Q311735,http://www.wikidata.org/entity/Q449245,http://www.wikidata.org/entity/Q201608,http://www.wikidata.org/entity/Q25278,http://www.wikidata.org/entity/Q275889,http://www.wikidata.org/entity/Q295364,http://www.wikidata.org/entity/Q309872,http://www.wikidata.org/entity/Q313481,http://www.wikida

### Get all data property of "basketball" (wd:Q5372)

In [342]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q5372 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1051'), ('label', 'PSH ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1417'), ('label', 'Encyclopædia Britannica Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1807'), ('label', 'Great Aragonese Encyclopedia ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1813'), ('label', 'short 

### Get all data property of "basketball"  (wd:Q5372) that do not contain ID

In [343]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q5372 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1036'), ('label', 'Dewey Decimal Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1245'), ('label', 'OmegaWiki Defined Meaning')]
[('property', 'http://www.wikidata.org/prop/direct/P1282'), ('label', 'OpenStreetMap tag or key')]
[('property', 'http://www.wikidata.org/prop/direct/P1813'), ('label', 'short name')]
[('property', 'http://www.wikidata.org/prop/direct/P1873'), ('label', 'maximum number of players')]
[('property', 'http://www.wikidata.org/prop/direct/P2047'), ('label', 'duration')]
[('property', 'http://www.wikidata.org/prop/direct/P2167'), ('label', 'UNSPSC Code')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P3984'), ('label', 'subreddit')]
[('property', 'http://www.wikidata.org/prop/direct/P48

### Get the class of "basketball" (wd:Q5372)

In [344]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q5372 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('class', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
2


### Get all object properties of "basketball"  (wd:Q5372)

In [346]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q5372 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('label', 'competition class')]
[('property', 'http://www.wikidata.org/prop/direct/P2283'), ('label', 'uses')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P2888'), ('label', 'exact match')]
[('property', 'http://www.wikidata.org/prop/direct/P2910'), ('label', 'icon')]
[('property', 'http://www.wikidata.org/prop/direct/P3095'), ('label', 'practiced by')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P443'), ('label', 'pronunciation audio')]
[('property', 'http://www.wikidata.o

### Get all the classes connected to "basketball"  (wd:Q5372)

In [347]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q5372 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q1489259'), ('label', 'superpower')]
[('objectClass', 'http://www.wikidata.org/entity/Q22936940'), ('label', 'competition class')]
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q1744559'), ('label', 'classification in sports')]
[('objectClass', 'http://www.wikidata.org/entity/Q99541706'), ('label', 'historical unrecognized state')]
[('objectClass', 'http://www.wikidata.org/entity/Q13406463'), ('label', 'Wikimedia list article')]
[('objectClass', 'http://www.wikidata.org/entity/Q1520223'), ('label', 'constitutional republic')]
[('objectClass', 'http://www.wikidata.org/entity/Q28640'), ('label', 'profession')]
[('objectClass', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q3624078'), ('label', 'sovereign state')]
[('objectClass', 'http://www.wikidata.org/entity/Q4167836'

### Get all the couples (object property - list of the classes of the objects that are connected to "basketball"  (wd:Q5372) through that object property)

In [348]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q5372 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objectsClasses', 'http://www.wikidata.org/entity/Q16695773,http://www.wikidata.org/entity/Q16695773,http://www.wikidata.org/entity/Q13406463,http://www.wikidata.org/entity/Q14204246'), ('labelsObjectsClasses', 'WikiProject,WikiProject,Wikimedia list article,Wikimedia project page')]
[('property', 'http://www.wikidata.org/prop/direct/P3095'), ('labelProperty', 'practiced by'), ('objectsClasses', 'http://www.wikidata.org/entity/Q28640'), ('labelsObjectsClasses', 'profession')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('labelProperty', 'different from'), ('objectsClasses', 'http://www.wikidata.org/entity/Q31629'), ('labelsObjectsClasses', 'type of sport')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('labelProperty', 'competition class'), ('objectsClasses', 'http://www.wikidata.org/entity/Q22936940,http://www.wikidata.org/entity/

### Get all the couples (object property - list of the  objects that are connected to "basketball"  (wd:Q5372) through that object property)

In [350]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q5372 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P5008'), ('labelProperty', 'on focus list of Wikimedia project'), ('objects', 'http://www.wikidata.org/entity/Q15304953,http://www.wikidata.org/entity/Q5460604'), ('labelsObjects', 'WikiProject African diaspora,Wikipedia:Vital articles')]
[('property', 'http://www.wikidata.org/prop/direct/P3095'), ('labelProperty', 'practiced by'), ('objects', 'http://www.wikidata.org/entity/Q3665646'), ('labelsObjects', 'basketball player')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('labelProperty', 'different from'), ('objects', 'http://www.wikidata.org/entity/Q231200'), ('labelsObjects', 'netball')]
[('property', 'http://www.wikidata.org/prop/direct/P2094'), ('labelProperty', 'competition class'), ('objects', 'http://www.wikidata.org/entity/Q46343203,http://www.wikidata.org/entity/Q2887217'), ('labelsObjects', "men's basketball,women's basketball")]
[('property', 'http://www.wikidata.org/prop/direct/P2283'), ('labelPropert

### Get the first 50 proprieties that connects subjects with the object "basketball player" (wd:Q3665646) such that the subjects are not literals

In [398]:
queryString = """
SELECT DISTINCT ?property ?labelProperty
WHERE 
{   
   ?subject ?property wd:Q3665646 .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?subject sc:name ?labelSubject
   
   FILTER(!isLiteral(?subject)) .   
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P106'), ('labelProperty', 'occupation')]
[('property', 'http://www.wikidata.org/prop/direct/P101'), ('labelProperty', 'field of work')]
[('property', 'http://www.wikidata.org/prop/direct/P1423'), ('labelProperty', 'template has topic')]
[('property', 'http://www.wikidata.org/prop/direct/P971'), ('labelProperty', 'category combines topics')]
[('property', 'http://www.wikidata.org/prop/direct/P3095'), ('labelProperty', 'practiced by')]
[('property', 'http://www.wikidata.org/prop/direct/P360'), ('labelProperty', 'is a list of')]
[('property', 'http://www.wikidata.org/prop/direct/P1269'), ('labelProperty', 'facet of')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('labelProperty', 'subclass of')]
[('property', 'http://www.wikidata.org/prop/direct/P921'), ('labelProperty', 'main subject')]
[('property', 'http://www.wikidata.org/pr

### Get all subjects connected with "sport" (wdt:P641) to basketball (wd:Q5372) such that the subject label contains "NBA Defensive Player"

In [392]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj 
WHERE {   
   ?sbj wdt:P641 wd:Q5372 ;
        sc:name ?labelSbj .
   FILTER (CONTAINS(?labelSbj, "NBA Defensive Player"))
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q505872'), ('labelSbj', 'WNBA Defensive Player of the Year Award')]
[('sbj', 'http://www.wikidata.org/entity/Q845884'), ('labelSbj', 'NBA Defensive Player of the Year Award')]
2


### Get the first 25 subjects connected with "sport" (wdt:P641) to "basketball" (wdt:Q5372) such that the label of the subject contains "Chicago Bulls"

In [401]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj 
WHERE {   
   ?sbj wdt:P641 wd:Q5372 ;
        sc:name ?labelSbj .
   FILTER (CONTAINS(?labelSbj, "Chicago Bulls"))
}
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q3667609'), ('labelSbj', '1985–86 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q3667614'), ('labelSbj', '1989–90 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q128109'), ('labelSbj', 'Chicago Bulls')]
[('sbj', 'http://www.wikidata.org/entity/Q3667626'), ('labelSbj', '1998–99 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q3667628'), ('labelSbj', '2000–01 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q371181'), ('labelSbj', '1999–2000 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q3465138'), ('labelSbj', '1967–68 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q48846'), ('labelSbj', '1968–69 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q969287'), ('labelSbj', '1966–67 Chicago Bulls season')]
[('sbj', 'http://www.wikidata.org/entity/Q3465314'), ('labelSbj', '1986–87 Chicago Bulls season')]
[('sbj', 'http://www

### Get the first 25 subjects connected with "sport" (wdt:P641) to "basketball" (wdt:Q5372) such that the label of the subject contains "Chicago Bulls" ordered descendant by label of the subject

In [403]:
queryString = """
SELECT DISTINCT ?sbj ?labelSbj 
WHERE {   
   ?sbj wdt:P641 wd:Q5372 ;
        sc:name ?labelSbj .
   FILTER (CONTAINS(?labelSbj, "Chicago Bulls"))
}ORDER BY DESC (?labelSbj)
LIMIT 25 
"""
print("Results")
x=run_query(queryString)

Results
[('sbj', 'http://www.wikidata.org/entity/Q4436738'), ('labelSbj', 'list of Chicago Bulls seasons')]
[('sbj', 'http://www.wikidata.org/entity/Q6566809'), ('labelSbj', 'list of Chicago Bulls broadcasters')]
[('sbj', 'http://www.wikidata.org/entity/Q12425605'), ('labelSbj', 'Template:Chicago Bulls 1997–98 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q12425662'), ('labelSbj', 'Template:Chicago Bulls 1996–97 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q12425887'), ('labelSbj', 'Template:Chicago Bulls 1995–96 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q12425733'), ('labelSbj', 'Template:Chicago Bulls 1992–93 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q6872880'), ('labelSbj', 'Template:Chicago Bulls 1991–92 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q6872879'), ('labelSbj', 'Template:Chicago Bulls 1990–91 NBA champions')]
[('sbj', 'http://www.wikidata.org/entity/Q56670521'), ('labelSbj', 'Game 4 of 1988 NBA Playof

### Get all data property of "2018–19 Chicago Bulls season" (wd:Q54866891)

In [368]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q640697 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get the class of "2018–19 Chicago Bulls season" (wd:Q54866891)

In [369]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q640697 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q13406463'), ('label', 'Wikimedia list article')]
1


### Get all object properties of "2018–19 Chicago Bulls season"  (wd:Q54866891)

In [388]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q1465181 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('label', 'instance of')]
[('property', 'http://www.wikidata.org/prop/direct/P641'), ('label', 'sport')]
[('property', 'http://www.wikidata.org/prop/direct/P973'), ('label', 'described at URL')]
3


### Get all the classes connected to "2018–19 Chicago Bulls season"  (wd:Q54866891)

In [371]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q640697 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q212434'), ('label', 'Olympic sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q31629'), ('label', 'type of sport')]
[('objectClass', 'http://www.wikidata.org/entity/Q55983715'), ('label', 'organisms known by a particular common name')]
3


### Get all the couples (object property - list of the  objects that are connected to "2018–19 Chicago Bulls season" (wd:Q54866891) through that object property)

In [383]:
queryString = """
SELECT DISTINCT ?obj #?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q52702755 wdt:P166 ?obj
 
   # Labels
   #?property sc:name ?labelProperty .      
   #?object sc:name ?labelObject
   
   #FILTER(!isLiteral(?object)) .   
}#GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get the value of the "inception" object propriety connected to "NBA Depensive Player of the Year Award" (wd:Q845884)

In [243]:
queryString = """
SELECT DISTINCT ?inception 
WHERE {   
   wd:Q845884 wdt:P571 ?inception .
   #?inception sc:name ?labelInception .
}
"""
print("Results")
x=run_query(queryString)

Results
[('inception', '1983-01-01T00:00:00Z')]
1


### Get all proprieties and object labels connected to "NBA Depensive Player of the Year Award" (wd:Q845884)

In [245]:
queryString = """
SELECT ?labelProperty ?labelObject
WHERE {   
   wd:Q845884 ?property ?object .
   ?property sc:name ?labelProperty .
   ?object sc:name ?labelObject .
}
"""
print("Results")
x=run_query(queryString)

Results
[('labelProperty', 'winner'), ('labelObject', 'Sidney Moncrief')]
[('labelProperty', 'instance of'), ('labelObject', 'award')]
[('labelProperty', 'winner'), ('labelObject', 'Alonzo Mourning')]
[('labelProperty', 'sport'), ('labelObject', 'basketball')]
[('labelProperty', 'winner'), ('labelObject', 'Michael Cooper')]
[('labelProperty', 'winner'), ('labelObject', 'Kevin Garnett')]
[('labelProperty', 'winner'), ('labelObject', 'Michael Jordan')]
[('labelProperty', 'instance of'), ('labelObject', 'class of award')]
[('labelProperty', 'winner'), ('labelObject', 'Hakeem Olajuwon')]
[('labelProperty', 'winner'), ('labelObject', 'David Robinson')]
[('labelProperty', 'winner'), ('labelObject', 'Marcus Camby')]
[('labelProperty', 'winner'), ('labelObject', 'Metta Sandiford-Artest')]
[('labelProperty', 'instance of'), ('labelObject', 'Wikimedia set index article')]
[('labelProperty', 'winner'), ('labelObject', 'Kawhi Leonard')]
[('labelProperty', 'winner'), ('labelObject', 'Joakim Noah')]

### Get all subjects and proprieties labels such that "NBA Depensive Player of the Year Award" (wd:Q845884) is the object

In [246]:
queryString = """
SELECT ?labelProperty ?labelSbj
WHERE {   
   ?sbj ?property wd:Q845884 .
   ?property sc:name ?labelProperty .
   ?sbj sc:name ?labelSbj .
}
"""
print("Results")
x=run_query(queryString)

Results
[('labelProperty', 'award received'), ('labelSbj', 'Sidney Moncrief')]
[('labelProperty', 'award received'), ('labelSbj', 'Alonzo Mourning')]
[('labelProperty', 'award received'), ('labelSbj', 'Michael Cooper')]
[('labelProperty', 'award received'), ('labelSbj', 'Kevin Garnett')]
[('labelProperty', 'award received'), ('labelSbj', 'Michael Jordan')]
[('labelProperty', 'award received'), ('labelSbj', 'Hakeem Olajuwon')]
[('labelProperty', 'award received'), ('labelSbj', 'David Robinson')]
[('labelProperty', 'award received'), ('labelSbj', 'Marcus Camby')]
[('labelProperty', 'award received'), ('labelSbj', 'Metta Sandiford-Artest')]
[('labelProperty', 'award received'), ('labelSbj', 'Kawhi Leonard')]
[('labelProperty', 'award received'), ('labelSbj', 'Joakim Noah')]
[('labelProperty', 'award received'), ('labelSbj', 'Alvin Robertson')]
[('labelProperty', 'award received'), ('labelSbj', 'Dennis Rodman')]
[('labelProperty', 'award received'), ('labelSbj', 'Dwight Howard')]
[('labelP

### Get the first 50 subclass (wdt:P31) of "NBA Defensive Player of the Year" (wd:Q845884)

In [396]:
queryString = """
SELECT ?subclass 
WHERE {   
   ?subclass wdt:P31 wd:Q845884 .   
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get the first 50 labels of subjects such that they are connected to "award" (wd:Q618779) as well as to "NBA Defensive Player of the Year" (wd:Q845884) and the label of the Subject contains "2000"

In [255]:
queryString = """
SELECT ?labelSub ?labelPred
WHERE {   
   ?sub ?pred wd:Q618779 ;
         sc:name ?labelSub .

   ?sub ?p wd:Q845884 .


   ?pred sc:name ?labelPred .   
   FILTER CONTAINS(?labelSub, "2000")   
}LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get all proprieties and objects connected to "members of sports team" (wdt:P54)

In [394]:
queryString = """ 
SELECT DISTINCT ?property ?object
WHERE {   
    wdt:P54 ?property ?object .    
}LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('object', 'http://www.w3.org/2002/07/owl#ObjectProperty')]
[('property', 'http://schema.org/name'), ('object', 'member of sports team')]
2


### Get the first 100 awards wons by "Micheal Jordan" (wd:Q41421)

In [417]:
queryString = """ 
SELECT ?award ?labelAward
WHERE
{
  wd:Q41421 wdt:P166 ?award .
  ?award sc:name ?labelAward .
}LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q1061233'), ('labelAward', "L'Équipe Champion of Champions")]
[('award', 'http://www.wikidata.org/entity/Q17144'), ('labelAward', 'Presidential Medal of Freedom')]
[('award', 'http://www.wikidata.org/entity/Q743122'), ('labelAward', 'Associated Press Athlete of the Year')]
[('award', 'http://www.wikidata.org/entity/Q137003'), ('labelAward', '50 Greatest Players in NBA History')]
[('award', 'http://www.wikidata.org/entity/Q1418009'), ('labelAward', 'NBA All-Rookie Team')]
[('award', 'http://www.wikidata.org/entity/Q1465181'), ('labelAward', 'NBA All-Defensive Team')]
[('award', 'http://www.wikidata.org/entity/Q1641970'), ('labelAward', 'John R. Wooden Award')]
[('award', 'http://www.wikidata.org/entity/Q1653207'), ('labelAward', 'Sports Illustrated Sportsperson of the Year')]
[('award', 'http://www.wikidata.org/entity/Q222047'), ('labelAward', 'NBA Most Valuable Player Award')]
[('award', 'http://www.wikidata.org/entity/Q2404133'), ('la

In [424]:
## TODO TRY AWARD RECEIVED, TRY EXPLORATION FROM OTHER SIDE FOR ALL EXPLORED OBJECTS,

In [ ]:
### "award" (wd:Q618779) "NBA Defensive Player of the Year" (wd:Q845884)

### Get all the incoming object proprieties of "NBA Defensive Player of the Year Award" (wd:Q845884)

In [13]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   ?object ?property wd:Q845884 .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
1


### Get all the incoming object proprieties of "Micheal Jordan" (wd:Q41421)

In [15]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   ?object ?property wd:Q41421 .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P112'), ('label', 'founded by')]
[('property', 'http://www.wikidata.org/prop/direct/P127'), ('label', 'owned by')]
[('property', 'http://www.wikidata.org/prop/direct/P1346'), ('label', 'winner')]
[('property', 'http://www.wikidata.org/prop/direct/P138'), ('label', 'named after')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('label', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P175'), ('label', 'performer')]
[('property', 'http://www.wikidata.org/prop/direct/P180'), ('label', 'depicts')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P22'), ('label', 'father')]
[('property', 'http://www.wikidata.org/prop/direct/P26'), ('label', 'spouse')]
[('property', 'http://www.wikidata.org/prop/direct/P301'), ('label', "category's main topic")]
[('property', 'http://www.wikidata.org/prop/direct/P3373'), ('label',

### Get all the incoming object proprieties of "2018-19 NBA Season" (wd:Q52702755)

In [16]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   ?object ?property wd:Q52702755 .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1269'), ('label', 'facet of')]
[('property', 'http://www.wikidata.org/prop/direct/P1344'), ('label', 'participant in')]
[('property', 'http://www.wikidata.org/prop/direct/P301'), ('label', "category's main topic")]
[('property', 'http://www.wikidata.org/prop/direct/P5249'), ('label', 'sports competition competed at')]
4


### Get all data property of "award" (wd:Q618779)

In [431]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q618779 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1014'), ('label', 'Art & Architecture Thesaurus ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1149'), ('label', 'Library of Congress Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P1296'), ('label', 'Gran Enciclopèdia Catalana ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2004'), ('label', 'NALT ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2163'), ('label', 'FAST ID')]
[('property', 'http://www.wikidata.org/prop/direct/P227'), ('label', 'GND ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2347'), ('label', 'YSO ID')]
[('property', 'http://www.wikidata.org/prop/direct/P244'), ('label', 'Library of Congress authority ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2581'), ('label', 'BabelNet ID')]
[('property', 'http://www.wikidata.org/prop/dire

### Get all data property of "award"  (wd:Q618779) that do not contain ID

In [433]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q618779 ?property ?object .
   ?property sc:name ?label .
   FILTER(isLiteral(?object)) .
   FILTER REGEX(?label, "^((?!ID).)*$") . 
}
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1149'), ('label', 'Library of Congress Classification')]
[('property', 'http://www.wikidata.org/prop/direct/P1225'), ('label', 'U.S. National Archives Identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('label', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P672'), ('label', 'MeSH tree code')]
[('property', 'http://www.wikidata.org/prop/direct/P7749'), ('label', 'Nomenclature for Museum Cataloging')]
5


### Get the class of "award" (wd:Q618779)

In [438]:
queryString = """
SELECT DISTINCT ?class ?label
WHERE {   
   wd:Q618779 wdt:P31 ?class .
   ?class sc:name ?label .     
}
"""
print("Results")
x=run_query(queryString)

Results
Empty


### Get all object properties of "award"  (wd:Q618779)

In [437]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   wd:Q618779 ?property ?object .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1151'), ('label', "topic's main Wikimedia portal")]
[('property', 'http://www.wikidata.org/prop/direct/P1343'), ('label', 'described by source')]
[('property', 'http://www.wikidata.org/prop/direct/P1382'), ('label', 'partially coincident with')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('label', "topic's main template")]
[('property', 'http://www.wikidata.org/prop/direct/P1687'), ('label', 'Wikidata property')]
[('property', 'http://www.wikidata.org/prop/direct/P1709'), ('label', 'equivalent class')]
[('property', 'http://www.wikidata.org/prop/direct/P18'), ('label', 'image')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P1963'), ('label', 'properties for this type')]
[('property', 'http://www.wikidata.org/prop/direct/P2354'), ('label', 'has list')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label

### Get all the classes connected to "award"  (wd:Q618779)

In [436]:
queryString = """
SELECT DISTINCT ?objectClass ?label
WHERE {   
   wd:Q618779 ?property ?object .
   ?object wdt:P31 ?objectClass .
   ?objectClass sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""
print("Results")
x=run_query(queryString)

Results
[('objectClass', 'http://www.wikidata.org/entity/Q16695773'), ('label', 'WikiProject')]
[('objectClass', 'http://www.wikidata.org/entity/Q22964785'), ('label', 'Wikidata property related to events')]
[('objectClass', 'http://www.wikidata.org/entity/Q61719274'), ('label', 'non-restrictive qualifier')]
[('objectClass', 'http://www.wikidata.org/entity/Q33104279'), ('label', 'philosophical concept')]
[('objectClass', 'http://www.wikidata.org/entity/Q96253971'), ('label', 'type of property')]
[('objectClass', 'http://www.wikidata.org/entity/Q15647814'), ('label', 'Wikimedia administration category')]
[('objectClass', 'http://www.wikidata.org/entity/Q30432511'), ('label', 'metacategory in Wikimedia projects')]
[('objectClass', 'http://www.wikidata.org/entity/Q33532284'), ('label', 'Wikimedia list of lists')]
[('objectClass', 'http://www.wikidata.org/entity/Q4167836'), ('label', 'Wikimedia category')]
[('objectClass', 'http://www.wikidata.org/entity/Q24017414'), ('label', 'second-orde

### Get all the couples (object property - list of the classes of the objects that are connected to "award"  (wd:Q618779) through that object property)

In [435]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?objectClass ; separator = ",")  AS ?objectsClasses) (GROUP_CONCAT(?labelObjectClass ; separator = ",")  AS ?labelsObjectsClasses)
WHERE 
{   
   wd:Q618779 ?property ?object .  
   ?object wdt:P31 ?objectClass .
   
   # Labels
   ?property sc:name ?labelProperty .      
   ?objectClass sc:name ?labelObjectClass .
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objectsClasses', 'http://www.wikidata.org/entity/Q24017414'), ('labelsObjectsClasses', 'second-order class')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objectsClasses', 'http://www.wikidata.org/entity/Q19887878'), ('labelsObjectsClasses', 'Wikimedia infobox template')]
[('property', 'http://www.wikidata.org/prop/direct/P6104'), ('labelProperty', 'maintained by WikiProject'), ('objectsClasses', 'http://www.wikidata.org/entity/Q16695773'), ('labelsObjectsClasses', 'WikiProject')]
[('property', 'http://www.wikidata.org/prop/direct/P6186'), ('labelProperty', 'category for eponymous categories'), ('objectsClasses', 'http://www.wikidata.org/entity/Q15647814,http://www.wikidata.org/entity/Q30432511'), ('labelsObjectsClasses', 'Wikimedia administration category,metacategory in Wikimedia projects')]
[('property', 'http://www.wikidata

### Get all the couples (object property - list of the  objects that are connected to "award"  (wd:Q618779) through that object property)

In [434]:
queryString = """
SELECT DISTINCT ?property ?labelProperty (GROUP_CONCAT(?object ; separator = ",")  AS ?objects) (GROUP_CONCAT(?labelObject; separator = ",")  AS ?labelsObjects)
WHERE 
{   
   wd:Q618779 ?property ?object .  
 
   # Labels
   ?property sc:name ?labelProperty .      
   ?object sc:name ?labelObject
   
   FILTER(!isLiteral(?object)) .   
}GROUP BY ?property ?labelProperty
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('labelProperty', 'has part'), ('objects', 'http://www.wikidata.org/entity/Q38033430'), ('labelsObjects', 'class of award')]
[('property', 'http://www.wikidata.org/prop/direct/P461'), ('labelProperty', 'opposite of'), ('objects', 'http://www.wikidata.org/entity/Q186361'), ('labelsObjects', 'punishment')]
[('property', 'http://www.wikidata.org/prop/direct/P1424'), ('labelProperty', "topic's main template"), ('objects', 'http://www.wikidata.org/entity/Q5626594'), ('labelsObjects', 'Template:Infobox award')]
[('property', 'http://www.wikidata.org/prop/direct/P6104'), ('labelProperty', 'maintained by WikiProject'), ('objects', 'http://www.wikidata.org/entity/Q18762205'), ('labelsObjects', 'WikiProject Awards and prizes')]
[('property', 'http://www.wikidata.org/prop/direct/P6186'), ('labelProperty', 'category for eponymous categories'), ('objects', 'http://www.wikidata.org/entity/Q8105429'), ('labelsObjects', 'Category:Wikip

### Get all the incoming object proprieties of "award" (wd:Q618779)

In [10]:
queryString = """
SELECT DISTINCT ?property ?label
WHERE {   
   ?object ?property wd:Q618779 .
   ?property sc:name ?label .
   FILTER(!isLiteral(?object)) .   
}
"""

print("Results")
x=run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1423'), ('label', 'template has topic')]
[('property', 'http://www.wikidata.org/prop/direct/P3712'), ('label', 'objective of project or action')]
[('property', 'http://www.wikidata.org/prop/direct/P101'), ('label', 'field of work')]
[('property', 'http://www.wikidata.org/prop/direct/P1056'), ('label', 'product or material produced')]
[('property', 'http://www.wikidata.org/prop/direct/P1269'), ('label', 'facet of')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('label', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P1382'), ('label', 'partially coincident with')]
[('property', 'http://www.wikidata.org/prop/direct/P166'), ('label', 'award received')]
[('property', 'http://www.wikidata.org/prop/direct/P180'), ('label', 'depicts')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('label', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P279'), ('label', 'subclass of')]

### Final Query
For this query I was not able to find a solution after the several trials that I have done. The information regarding the year which a player won the "NBA Defensive Player of the Year" is present in wikidata with prefix "p". The only information I can get is the list of players who won such an award but not when they won it.

Final query for this task

In [3]:
queryString = """
SELECT DISTINCT ?player ?labelPlayer
WHERE 
{   
    ?player wdt:P166 wd:Q845884 ;
            sc:name ?labelPlayer .
}
"""
print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q177928'), ('labelPlayer', 'Sidney Moncrief')]
[('player', 'http://www.wikidata.org/entity/Q310968'), ('labelPlayer', 'Alonzo Mourning')]
[('player', 'http://www.wikidata.org/entity/Q971538'), ('labelPlayer', 'Michael Cooper')]
[('player', 'http://www.wikidata.org/entity/Q189240'), ('labelPlayer', 'Kevin Garnett')]
[('player', 'http://www.wikidata.org/entity/Q41421'), ('labelPlayer', 'Michael Jordan')]
[('player', 'http://www.wikidata.org/entity/Q273256'), ('labelPlayer', 'Hakeem Olajuwon')]
[('player', 'http://www.wikidata.org/entity/Q211876'), ('labelPlayer', 'David Robinson')]
[('player', 'http://www.wikidata.org/entity/Q361134'), ('labelPlayer', 'Marcus Camby')]
[('player', 'http://www.wikidata.org/entity/Q295322'), ('labelPlayer', 'Metta Sandiford-Artest')]
[('player', 'http://www.wikidata.org/entity/Q926248'), ('labelPlayer', 'Kawhi Leonard')]
[('player', 'http://www.wikidata.org/entity/Q311735'), ('labelPlayer', 'Joakim Noah')]